In [ ]:
from block_sync_class import *
from bokeh.plotting import figure, show
import pickle
import time


In [ ]:
blockdict = {}
for i in range(8):
    blockdict[f'block_{i}'] = BlockSync(animal_num='4',
                                        experiment_date='18_07_2021',
                                        block_num=str(i),
                                        path_to_animal_folder=r'D:\\experiments\\')

In [ ]:
for i in range(1):
    block = blockdict[f'block_{i}']
    block.handle_arena_files()
    block.handle_eye_videos()
    block.synchronize_arena_timestamps()
    block.create_arena_brightness_df(250, export=True)

In [ ]:
for i in range(1):
    block = blockdict[f'block_{i}']
    block.import_open_ephys_events()
    block.synchronize_block()
    block.update_arena_bdf()
    block.create_eye_brightness_df(30)
    block.validate_eye_synchronization()

In [ ]:
block=blockdict[f'block_0']
block.synchronize_by_led_blink(threshold_value_for_eyes=30)
block.block_plot()

In [ ]:
block = blockdict['block_0']
block.arena_vidnames[1]

In [ ]:
for i in range(1):
    block = blockdict[f'block_{i}']
    vid_to_use = None
    if i in [0,1,4,7]:
        vid_to_use = block.arena_vidnames[1]
        print(f'synchronizing with {vid_to_use}')
    block.synchronize_by_led_blink(arena_vid_to_use=vid_to_use)

In [ ]:
block_0 = blockdict.pop('block_0')
block_4 = blockdict.pop('block_4')

In [ ]:
block_4.validate_arena_synchronization_bdf()

In [ ]:
blockdict['block_0'] = block_0

In [ ]:
for i in range(8):
    if f'block_{i}' in blockdict.keys():
        block = blockdict[f'block_{i}']
    else:
        continue
    block.create_synced_videos_validated()
    block.export_everything()

In [ ]:
for block in blocklist:
    start = time.time()
    block.create_arena_brightness_df(threshold_value=255, export=True)
    end = time.time()
    print(f'this one took {end - start} seconds')

In [ ]:
for block in blocklist:
    #block.validate_arena_synchronization()
    block.import_open_ephys_events()
    block.synchronize_block()
    block.update_arena_bdf()
    block.create_eye_brightness_df(threshold_value=30)
    block.synchronize_by_led_blink()
    
    file_to_save_to = open(block.block_path / 'BlockSync.pickle','wb')
    pickle.dump(block, file_to_save_to)
    file_to_save_to.close()
    print(f'block number {block.block_num} saved')


##### PHASE 2 #####

In [ ]:
from block_sync_class import *
from bokeh.plotting import figure, show
import pickle
import time
from bokeh.models import HoverTool
from block_sync_class import *
from bokeh.plotting import figure, show
from matplotlib import pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
blockdict = {}
for i in range(8):
    blockdict[f'block_{i}'] = BlockSync(animal_num='4',
                                        experiment_date='18_07_2021',
                                        block_num=str(i),
                                        path_to_animal_folder=r'D:\\experiments\\')

In [ ]:
del blockdict['block_4']
blockdict

In [ ]:
blockdict['block_2'].block_plot()


In [ ]:
for num in range(8):
    if num == 1:
        print(f'working on block {num}')
        block = blockdict[f'block_{num}']
        block.handle_arena_files()
        block.handle_eye_videos()
        block.synchronize_arena_timestamps(get_only_anchor_vid=True)
        block.import_everything()
        block.read_dlc_data()
        block.get_image_events_frames()
        #block.led_speed_analysis(plot_graph=False,speed_threshold=0.007)
        #block.pupil_speed_graph(smooth_velocities=True)
        #block.block_plot()

In [ ]:
for num in range(8):
    if num!= 4:
        block = blockdict[f'block_{num}']
        block.collect_saccade_events_full_dynamics(threshold=1.2)

In [ ]:
blocklist = []
for num in range(8):
    if num not in [4,3]:
        block = blockdict[f'block_{num}']
        if block.saccade_list is not None:
            print(f'block_{block.block_num} is good to go')
            blocklist.append(block)
blocklist

In [ ]:
saccade_raster_list = []
no_movement_raster_list = []
last_frame = 0
connections = []
for block in blocklist:
    b_raster = np.array(pd.DataFrame(block.saccade_list).starts.values) + last_frame
    b_no_movements = np.array(block.no_movement_frames) + last_frame
    saccade_raster_list.append(b_raster)
    no_movement_raster_list.append(b_no_movements)
    connections.append(last_frame)
    last_frame += block.le_video_sync_df.index.values[-1]
flat_raster = [s for sublist in saccade_raster_list for s in sublist]
flat_no_movement_raster = [s for sublist in no_movement_raster_list for s in sublist]
no_movement_array = np.sort(np.array(flat_no_movement_raster))
raster_array = np.sort(np.array(flat_raster))

p = Path(r'D:\MarkS3\Documents\My_Writing\Sagol_Phd_submission\Graphs')
#np.save(p / 'saccade_raster.npy',raster_array)

In [ ]:
#block = blockdict['block_0']
#saccade_df = pd.DataFrame(block.saccade_list)
#s_frames = saccade_df.starts.values
x_axis = list(range(int(raster_array[-1])))
x_axis = np.array(list(range(int(raster_array[-1]))))*(1/60)
fig, axes = plt.subplots(figsize=[8,1.2])
fig.set_facecolor('xkcd:white')
axes.plot(x_axis,[0]*len(x_axis),'white')
axes.vlines(raster_array*(1/60),-1,1,linewidth=0.5)
axes.vlines(no_movement_array*(1/60),-1,0.2,linewidth=0.1, color='green')
axes.vlines(np.array(connections)*(1/60),-1,1,linewidth=2,color='red')
axes.set_ylim(0,1)
axes.set_xlim(0,1300)
plt.xlabel('Time [Seconds]',fontsize=12)
plt.yticks([])
plt.tight_layout()
#plt.savefig(p / 'saccade_raster_top.png', quality=100)

In [ ]:
block.no_movement_frames

In [ ]:
isi = np.diff(raster_array*(1/60),prepend=1)
isi = isi[isi!=0]
fig, axes = plt.subplots()
fig.set_facecolor('xkcd:white')

_ = plt.hist(isi,np.linspace(0,10,100))

In [ ]:
def produce_saccade_plot(blockdict):
    lines = []
    for key in blockdict.keys():
        block = blockdict[key]
        try:
            saccade_df = pd.DataFrame(block.saccade_list)
        except AttributeError:
            print(f'block_{block.block_num} has no validated saccades')
            continue
        lengths = saccade_df.ends - saccade_df.starts
        saccade_df.insert(2,'length',lengths)
        cs_df = saccade_df.query('length < 30').dropna(subset=['dynamics','magnitude'])
        b_lines = cs_df.dynamics.values
        lines.append(b_lines)
    lines = np.concatenate(lines)

    # for i in range(len(lines)):
    #     for num,x in enumerate(lines[i][0:25]):
    #         lines[i][num] = x*-1
    lines_arr = [np.array(x) for x in lines]
    lines_stack = np.stack(lines_arr)
    mean_line = np.mean(lines_stack,axis=0)
    median_line = np.median(lines_stack, axis=0)
    # plot the saccade dynamics graph
    fig, axes = plt.subplots(figsize=[10,10])
    fig.set_facecolor('xkcd:white')
    plt.xlabel('frames', fontsize=20)
    plt.ylabel('Z score location inside saccade', fontsize=20)
    axes.set_title('Saccade Dynamics across trials',fontsize=25)

    for line in lines:
        axes.plot(line,'grey',alpha=0.2)
    axes.vlines(25,-1,1,linewidth=3,linestyles='dashed',label='saccade speed threshold crossing')
    axes.plot(mean_line,'r',label='Mean Saccade profile',linewidth=4)
    _ = axes.legend(fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    axes.xaxis.set_major_locator(MultipleLocator(5))
    #axes.xaxis.set_minor_locator(MultipleLocator())
    axes.tick_params(which='minor', length=5)
    fig.show()
    save = input('Save the figure? (y/n)')
    if save == 'n':
        return None
    elif save == 'y':
        name = input('name of the graph?')
        p = Path('D:\MarkS3\Documents\My_Writing\Sagol_Phd_submission\Graphs')
        plt.savefig(rf'{p}\{name}.png')

In [ ]:
produce_saccade_plot(blockdict=blockdict)



In [ ]:
ploc_l = []
mc_inds = []
mc_vals = []
is_corrected = []
for l in cs_df.dynamics.values:
    peak_loc = l.index(max(l))
    if peak_loc == len(l)-1:
        is_corrected.append(False)
    else:
        is_corrected.append(True)
    max_correction_value = min(l[peak_loc:])
    max_correction_ind = l[peak_loc:].index(max_correction_value)
    ploc_l.append(peak_loc)
    mc_inds.append(max_correction_ind)
    mc_vals.append(max_correction_value)

In [ ]:
block = blockdict['block_0']
pd.DataFrame(block.saccade_list)

In [ ]:
saccade_df = pd.DataFrame(block.saccade_list)

In [ ]:
lengths = saccade_df.ends - saccade_df.starts
if 'length' not in saccade_df.columns:
    saccade_df.insert(2,'length',lengths)
cs_df = saccade_df.query('length < 30').dropna(subset=['r_dynamics','magnitude', 'velocity'])
lines = cs_df.velocity.values
p=Path(r'D:\MarkS3\Documents\My_Writing\Sagol_Phd_submission\Graphs')
cs_df.to_csv(p / 'csdf.csv')

In [ ]:
len(blockdict['block_0'].synced_videos_validated) / 60
7400 / 60

In [ ]:
re_x_zscores = block.re_video_sync_df.center_x# - np.mean(block.re_video_sync_df.center_x)) / block.re_video_sync_df.center_x.std()
re_y_zscores = block.re_video_sync_df.center_y#- np.mean(block.re_video_sync_df.center_y)) / block.re_video_sync_df.center_y.std()
fig, axes = plt.subplots(2,1,figsize=(7,3))
fig.set_facecolor('xkcd:white')
region = list(range(9000,11000))
#region = list(range(7400,9400))
x_axis = np.array(region) - region[0]
x_axis = x_axis*(1/60)
axes[0].plot(x_axis,re_x_zscores[region])
axes[0].set_xlim(0,33)
#axes[0].set_ylim(-6,1)
axes[0].set_xticks([])
axes[1].plot(x_axis,re_y_zscores[region])
axes[1].set_xlim(0,33)
fig.text(0.05,0.5,'Pupil Position [Pixels]',ha='center',va='center',rotation='vertical')
fig.text(0.5,0.05,'Time [Seconds]',ha='center',va='center',rotation='horizontal')
fig.text(0.91,0.67,'X pos.',rotation='horizontal',fontsize=15)
fig.text(0.91,0.30,'Y pos.',rotation='horizontal',fontsize=15)
plt.subplots_adjust(bottom=0.16)
plt.savefig(p / 'xy_position_for_raster_plot.png',quality=100)

In [ ]:
block= blockdict['block_2']
fig,axes = plt.subplots()
fig.set_facecolor('xkcd:white')
region = list(range(878,940))
le_trajectory = (block.le_video_sync_df.center_x.values[region] - np.min(block.le_video_sync_df.center_x.values[region])) /(np.max(block.le_video_sync_df.center_x.values[region])-np.min(block.le_video_sync_df.center_x.values[region]))
re_trajectory = (block.re_video_sync_df.center_x.values[region] - np.min(block.re_video_sync_df.center_x.values[region])) /(np.max(block.re_video_sync_df.center_x.values[region])-np.min(block.re_video_sync_df.center_x.values[region]))
x_axis = np.array(region) - region[0]
x_axis = x_axis *(1/60)
axes.plot(x_axis,le_trajectory)
axes.plot(x_axis,re_trajectory)
axes.set_ylabel('normalized pupil X axis pos.')
axes.set_xlabel('Time [Seconds]')
plt.savefig(p / 'single_saccade.png',quality=100)

In [ ]:
lines_arr = [np.array(x) for x in lines]
normalized_lines = []
for x in lines_arr:
    l = [(j - np.min(x)) / (np.max(x)-np.min(x)) for j in x]
    normalized_lines.append(l)
lines_arr = [np.array(x) for x in normalized_lines]

#lines_arr = [np.cumsum(x) for x in lines_arr]

In [ ]:
lines_stack = np.stack(lines_arr)
mean_line = np.nanmean(lines_stack,axis=0)
cutlines = []
for line in lines_stack:
    diff_line = np.diff(line,prepend=1)
    deflection1 = np.argwhere(diff_line[25:]<0)
    if len(deflection1) >0:
        cut = int(deflection1[0][0] + 28)
        cutlines.append(line[:cut])
cutlines = np.array(cutlines)

In [ ]:
median_line = np.nanmedian(lines_stack, axis=0)
# plot the saccade dynamics graph
fig, axes = plt.subplots(figsize=[5,5])
fig.set_facecolor('xkcd:white')
plt.xlabel('Time [Seconds]', fontsize=15)
plt.ylabel('Normalized Velocity', fontsize=15)
x_axis = ['{:.2f}'.format(x) for x in list(((np.array(range(15)))-5)*(1/60))]
#axes.set_title('Saccade Dynamics across trials',fontsize=25)
for line in cutlines:
    axes.plot(line[20:],'grey',alpha=0.2)
axes.vlines(5,0,1.1,linewidth=3,linestyles='dashed',label='saccade speed threshold crossing')
axes.plot(x_axis,mean_line[20:35],'r',label='Mean Saccade profile',linewidth=4)
#axes.plot(median_line,'b',label='Mean Saccade profile',linewidth=4)
_ = axes.legend(fontsize=11,loc='upper left')
plt.xticks([x_axis[0], x_axis[5],x_axis[10],x_axis[14]],fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.ylim(0,1.2)
plt.xlim(-0.08)
fig.show()
p=Path(r'D:\MarkS3\Documents\My_Writing\Sagol_Phd_submission\Graphs')
plt.savefig(p / 'saccade_velocity_plot.png')

In [ ]:
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [ ]:
block.le_video_sync_df.Time[region].values[0]

In [ ]:
region = list(range(10274,10876))
fig, ax = plt.subplots(figsize=[6,4.25])
fig.set_facecolor('xkcd:white')

plt.xlabel('Pupil pos. X',fontsize=15)
plt.ylabel('Pupil pos. Y',fontsize=15)
plt.xticks([])
plt.yticks([])
plt.scatter(block.le_video_sync_df.center_x.iloc[region].values,
            block.le_video_sync_df.center_y[region].values,
            c=block.le_video_sync_df.Time[region],
            cmap='inferno_r',
            alpha=0.7,
            label='Left eye',
            s=13)
cb = plt.colorbar(shrink=0.6)
cb.set_ticks([192,202])
cb.ax.set_yticklabels(['0','10'])
bar = AnchoredSizeBar(ax.transData, 10, '10 pixels', 4, borderpad=0.2, frameon=False)
ax.add_artist(bar)
cb.set_label(label='Time [seconds]',fontsize=15)
plt.plot(block.le_video_sync_df.center_x.iloc[region].values,
         block.le_video_sync_df.center_y[region].values,linewidth=0.8)
plt.axis('equal')
fig.tight_layout()

In [ ]:
p=Path(r'D:\MarkS3\Documents\My_Writing\Sagol_Phd_submission\Graphs')
#plt.savefig(p / 'saccade_trajectory_plot.png', quality=100)

In [ ]:
block.block_plot()

In [ ]:
fig, axs = plt.subplots(4,1,figsize=[5,8])
fig.set_facecolor('xkcd:white')
axs = axs.flatten()
inds = [
    [1927,1927+180],
    [5530,5530+180],
    [4950,5000+250],
    [5400,5400+250]
]
block_nums = ['0','0','2','2']
for i in range(4):
    block = blockdict[f'block_{block_nums[i]}']
    frames = range(inds[i][0],inds[i][1])
    x_axis = np.array(block.re_video_sync_df.ellipse_size.loc[frames].index.values)
    x_axis = x_axis-x_axis[0]
    x_axis = x_axis * (1/60)
    axs[i].plot(x_axis,
                block.re_video_sync_df.ellipse_size.loc[frames].values)
    #axs[i].vlines(x_axis[10],min(block.re_video_sync_df.ellipse_size.loc[frames].values),
     #             max(block.re_video_sync_df.ellipse_size.loc[frames].values),color='green')
    axs[i].set_xticks([x_axis[0],x_axis[len(x_axis)//2],x_axis[-1]])
    #axs[i].set_xtickslabels([x_axis[0],x_axis[len(x_axis)//2],x_axis[-1]])
fig.text(0.05,0.5,'Pupil Diameter [Pixels]',ha='center',va='center',rotation='vertical')
fig.text(0.5,0.05,'Time [Seconds]',ha='center',va='center',rotation='horizontal')
plt.subplots_adjust(bottom=0.1,left=0.17)
plt.savefig(p / 'pupil_Dynamics.png',quality=100)

In [ ]:
np.save(p / 'saccade.npy',np.array(pd.DataFrame(block.saccade_list).starts.values))

In [ ]:
lx = block.le_video_sync_df.center_x.values
ly = block.le_video_sync_df.center_y.values
rx = block.re_video_sync_df.center_x.values
ry = block.re_video_sync_df.center_y.values
diff_dict = {
    'lx':np.diff(lx).astype(float),
    'ly':np.diff(ly).astype(float),
    'rx':np.diff(rx).astype(float),
    'ry':np.diff(ry).astype(float),
}
l_e_dist = np.sqrt(diff_dict['lx']**2 + diff_dict['ly']**2)
r_e_dist = np.sqrt(diff_dict['rx']**2 + diff_dict['ry']**2)


In [ ]:
bokeh_fig = figure(title='Pupil center speed',
                   x_axis_label='Linear Frames',
                   y_axis_label='euclidean speed',
                   plot_width=1500,
                   plot_height=700
                   )
x_axis = block.synced_videos_validated.Arena_TTL.values - block.arena_first_ttl_frame
bokeh_fig.line(x_axis, l_e_dist, legend_label='Left Eye Speed', line_width=1.5, line_color='blue')
bokeh_fig.line(x_axis, -r_e_dist, legend_label='Inverse Right Eye Speed', line_width=1.5, line_color='red')
#bokeh_fig.vbar(x=block.no_movement_frames, width=1, bottom=-4, top=-1, alpha=0.2, color='green')
bokeh_fig.xaxis.ticker = [x for x in x_axis]
show(bokeh_fig)

In [ ]:
block1 = blockdict['block_1']
block1.block_plot()





In [ ]:
re_dfs_to_concat = []
le_dfs_to_concat =[]
keys = []
breaks = [0]
concat_image_on_frames = []
concat_image_off_frames = []
for num in range(8):
    if num != 4:
        block = blockdict[f'block_{num}']
        keys.append(f'block_{block.block_num}')
        re_dfs_to_concat.append(block.re_video_sync_df)
        le_dfs_to_concat.append(block.le_video_sync_df)
        concat_image_on_frames.append(list(np.array(block.stim_on_arena_frames) + breaks[-1]))
        concat_image_off_frames.append(list(np.array(block.stim_off_arena_frames) + breaks[-1]))
        breaks.append(len(block.re_video_sync_df)+breaks[-1])
flat_image_on_frames = [x for sublist in concat_image_on_frames for x in sublist]
flat_image_off_frames = [x for sublist in concat_image_off_frames for x in sublist]
images_x_axis = [10]*len(flat_image_on_frames)

In [ ]:
re_mega_df = pd.concat(re_dfs_to_concat, keys=keys)
le_mega_df = pd.concat(le_dfs_to_concat, keys=keys)

In [ ]:
print(breaks)
trials_start = []
for i in range(8):
    if i != 4:
        trials_start.append(breaks[i] + blockdict[f'block_{i}'].synced_videos_validated['Arena_TTL'][0])
print(trials_start)

In [ ]:
#Extract the data columns and calculate Z-score across all blocks
le_ellipses_z = (le_mega_df.ellipse_size - le_mega_df.ellipse_size.mean()) / le_mega_df.ellipse_size.std()
re_ellipses_z = (re_mega_df.ellipse_size - re_mega_df.ellipse_size.mean()) / re_mega_df.ellipse_size.std()
le_x_zscores = (le_mega_df.center_x - np.mean(le_mega_df.center_x)) / le_mega_df.center_x.std()
le_y_zscores = (le_mega_df.center_y - np.mean(le_mega_df.center_y)) / le_mega_df.center_y.std()
re_x_zscores = (re_mega_df.center_x - np.mean(re_mega_df.center_x)) / re_mega_df.center_x.std()
re_y_zscores = (re_mega_df.center_y - np.mean(re_mega_df.center_y)) / re_mega_df.center_y.std()
x_axis = range(len(le_ellipses_z))

In [ ]:
# plot everything
bokeh_fig = figure(title='Pupil combined metrics',
                   x_axis_label='Linear Frames',
                   y_axis_label=' Z scores',
                   plot_width=1500,
                   plot_height=700
                   )
bokeh_fig.add_tools(HoverTool())
bokeh_fig.line(x_axis, le_ellipses_z + 7, legend_label='Left Eye diameter', line_width=1.5, line_color='blue')
bokeh_fig.line(x_axis, le_x_zscores + 14, legend_label='Left Eye x position', line_width=1, line_color='cyan')
bokeh_fig.line(x_axis, le_y_zscores, legend_label='Left Eye y position', line_width=1, line_color='green')
bokeh_fig.line(x_axis, re_ellipses_z + 7, legend_label='Right Eye diameter', line_width=1.5, line_color='red')
bokeh_fig.line(x_axis, re_x_zscores + 14, legend_label='Right Eye x position', line_width=1, line_color='orange')
bokeh_fig.line(x_axis, re_y_zscores, legend_label='Right Eye y position', line_width=1, line_color='pink')
bokeh_fig.cross(trials_start, [-5,-5,-5,-5,-5,-5,-5], size=20, color="navy", alpha=0.5)
bokeh_fig.vbar(x=flat_image_on_frames, width=1, bottom=-4, top=17, alpha=0.5, color='green')
bokeh_fig.vbar(x=flat_image_off_frames, width=1, bottom=-4, top=17, alpha=0.5, color='red')
#bokeh_fig.cross(flat_image_on_frames, images_x_axis, size=20, color="red", alpha=0.5)
bokeh_fig.line(x_axis, np.gradient(re_ellipses_z), legend_label='RE Gradient', line_width=1, line_color='green')
show(bokeh_fig)

In [ ]:
for i in range(8):
    if i != 4:
        block = blockdict[f'block_{i}']
        print(block.stim_on_arena_frames)

In [ ]:
first_image_on = [x[0] for x in concat_image_on_frames]
print(first_image_on)
first_supposed_to_be = []
for i in range(8):
    if i != 4:
        first_supposed_to_be.append(blockdict[f'block_{i}'].arena_first_ttl_frame)
len(breaks)

In [ ]:
le_mega_df

In [ ]:



### PHASE 3 ###
from bokeh.palettes import Category20c

In [ ]:
Category20c

In [ ]:
block = blockdict['block_1']
#for each block:

In [ ]:
# import the dlc_csvs and collect the data from all videos in a dataframe
m_csvs = list((block.block_path /'gaze_vector').glob('*.csv'))
csv_dict = {}
#create the csv dict to iterate over
for i in range(len(m_csvs)):
    csv_dict[f'csv_{i}'] = pd.read_csv(m_csvs[i],header=1)

# iterate and sort the data from the dlc into location_data
location_data = {}
uncertainty_thr = 0.5
for i in range(len(m_csvs)):
    data = csv_dict[f'csv_{i}']
    # import the dataframe and convert it to floats
    data = data.iloc[1:].apply(pd.to_numeric)
    # sort the pupil elements to x and y, with p as probability
    led_elements = np.array([x for x in data.columns if 'bodyparts' not in x])
    led_xs = data[led_elements[np.arange(0, len(led_elements), 3)]]
    led_ys = data[led_elements[np.arange(1, len(led_elements), 3)]]
    led_ps = data[led_elements[np.arange(2, len(led_elements), 3)]]
    # rename dataframes for masking with p values of bad points:
    led_ps = led_ps.rename(columns=dict(zip(led_ps.columns, led_xs.columns)))
    led_ys = led_ys.rename(columns=dict(zip(led_ys.columns, led_xs.columns)))
    good_points = led_ps > uncertainty_thr
    led_xs = led_xs[good_points]
    led_ys = led_ys[good_points]
    #Data collection into dict
    location_data[f'led_xs_{i}'] = led_xs
    location_data[f'led_ys_{i}'] = led_ys
    location_data[f'led_ps_{i}'] = led_ps

In [ ]:
dict_for_df = {}
for i in range(len(m_csvs)):
    dfx = location_data[f'led_xs_{i}']
    dfy = location_data[f'led_ys_{i}']
    new_columns_x = dfx.columns.values + f'_x_{i}'
    new_columns_y = dfy.columns.values + f'_y_{i}'
    dict_for_df[f'led_xs_{i}'] = dfx.rename(columns=dict(zip(dfx.columns.values, list(new_columns_x))))
    dict_for_df[f'led_ys_{i}'] = dfy.rename(columns=dict(zip(dfy.columns.values, list(new_columns_y))))
df_list = [dict_for_df[x] for x in dict_for_df.keys()]
movement_df = pd.concat(df_list,1)

In [ ]:
good_rows = []
bad_rows = []
for row in tqdm(range(0, len(movement_df))):
    count = 0
    for column in movement_df.columns:
        if not np.isnan(movement_df[f'{column}'].iloc[row]):
            count+=1
    if count > 20:
        good_rows.append(row)
    else:
        bad_rows.append(row)

In [ ]:
all_diffs_x = []
all_diffs_y = []
for col in movement_df.columns:
    y = movement_df[col].values
    x = list(range(len(movement_df)))
    dydx = np.diff(y)/np.diff(x)
    if 'x' in str(col):
        all_diffs_x.append(dydx)
    elif 'y' in str(col):
        all_diffs_y.append(dydx)
    else:
        print('weird, try again')
all_diffs_x = np.array(all_diffs_x)
all_diffs_y = np.array(all_diffs_y)
all_diffs_dict = {'y':all_diffs_y,
                  'x':all_diffs_x}

In [ ]:
mean_speed_dict = {
    'x':[],
    'y':[]
}
for key in ['x','y']:
    for i in range(len(all_diffs_dict[key][0])):
        # find the non nan values to work with:
        non_nans = []
        for j in range(len(all_diffs_dict[key])):
            if not np.isnan(all_diffs_dict[key][j][i]):
                non_nans.append(j)
        if len(non_nans) != 0:
            mean = np.sum(all_diffs_dict[key][non_nans,i])/len(non_nans)
            mean_speed_dict[key].append(mean)
        else:
            mean_speed_dict[key].append(np.nan)
            if np.isnan(mean):
                print('try again')
                print(i)
                print(non_nans)
                break

In [ ]:
x_sqrd = np.power(np.array(mean_speed_dict['x']),np.array([2]))
y_sqrd = np.power(np.array(mean_speed_dict['y']),np.array([2]))
euclidean_speed = np.sqrt(x_sqrd + y_sqrd)


In [ ]:
low_speed_segments = euclidean_speed < 0.02
low_speed_segments = medfilt(low_speed_segments,15)
bokeh_fig = figure(title='led movements per frame',
                   x_axis_label='Linear Frames',
                   y_axis_label='location in frame',
                   plot_width=1500,
                   plot_height=700
                   )
mean_speed = euclidean_speed
bokeh_fig.line(
    list(range(len(mean_speed))),
    np.abs((mean_speed - np.mean(mean_speed[~np.isnan(mean_speed)])) / np.std(np.array(mean_speed)[~np.isnan(mean_speed)])),
    legend_label='speed',
    line_width=0.3,
    line_color='black')
for num, col in enumerate(movement_df.columns):
    if 'y' in col:
        try:
            color = Category20c[20][num]
        except IndexError:
            color = 'green'
    elif 'x' in col:
        try:
            color = Category20c[20][-num]
        except IndexError:
            color = 'red'
    bokeh_fig.line(movement_df[col].index,
                   np.abs((movement_df[col].values - movement_df[col].mean()) / movement_df[col].std()),
                   legend_label=col,
                   line_width=1.5,
                   line_color=color)

bokeh_fig.vbar(x=np.argwhere(low_speed_segments)[:,0], width=1, bottom=-4, top=-1, alpha=0.2, color='green')
show(bokeh_fig)


In [ ]:
bokeh_fig = figure(title='led movements per frame',
                   x_axis_label='Linear Frames',
                   y_axis_label='location in frame',
                   plot_width=1500,
                   plot_height=700
                   )

bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_y_0.values, legend_label='y0', line_width=1.5, line_color='blue')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_x_0.values, legend_label='x0', line_width=1.5, line_color='cyan')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_y_1.values, legend_label='y1', line_width=1.5, line_color='red')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_x_1.values, legend_label='x1', line_width=1.5, line_color='orange')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_y_2.values, legend_label='y2', line_width=1.5, line_color='purple')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_x_2.values, legend_label='x2', line_width=1.5, line_color='pink')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_y_3.values, legend_label='y3', line_width=1.5, line_color='green')
bokeh_fig.line(movement_df.FL_y_0.index, movement_df.FL_x_3.values, legend_label='x3', line_width=1.5, line_color='black')
bokeh_fig.vbar(x=good_rows, width=1, bottom=-4, top=17, alpha=0.5, color='green')
#bokeh_fig.line(movement_df.FL_y_0.index, mean_speeds_per_frame, legend_label='speed', line_width=1.5, line_color='black')
show(bokeh_fig)


In [ ]:
import pathlib
import os
import pandas as pd
import shutil

In [ ]:
dlc_path = Path(r'D:\experiments\Animal_4\18_07_2021\DLC filtered\arima filtered')
dlc_file_list = list(dlc_path.glob('*.csv'))
for file in dlc_file_list:
    dlc_filename = file.name[:-50]
    dlc_filename = dlc_filename.replace('-','_')
    print(dlc_filename)

In [ ]:
for i in range(8):
    if i != 4:
        block = blockdict[f'block_{i}']
        block_destination = block.block_path / 'gaze_vector'
        destination_code = block.arena_vidnames[0][5:-4]

        print(destination_code)
        for file in dlc_file_list:
            dlc_filename = file.name[:-50]
            dlc_filename = dlc_filename.replace('-','_')
            if destination_code in dlc_filename:
                #print(f'would have copied {dlc_filename} into {block_destination/dlc_filename} because the code is {destination_code}')
                dlc_filename = str(dlc_filename + '.csv')
                shutil.copy(file, block_destination/dlc_filename)

In [ ]:
for i in range(8):
    if i !=4:
        block = blockdict[f'block_{i}']
        dest = block.block_path / 'gaze_vector'
        dest_files = list(dest.glob('*'))
        for file in dest_files:
            os.remove(file)

In [ ]:
from scipy.signal import medfilt
from matplotlib import pyplot as plt
low_speed_segments = euclidean_speed < 0.02
fig = plt.figure(figsize=(15,5))
plt.plot(range(len(low_speed_segments)),low_speed_segments)

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.plot(range(len(low_speed_segments)),medfilt(low_speed_segments,1))

In [ ]:
low_speed_segments = euclidean_speed < 0.02
np.shape(low_speed_segments)

In [ ]:
def get_closest_frame(timestamp, vid_timeseries, report_acc=None):
    """
    This function extracts a frame from a series so that it is as close as possible to a given timestamp
    :param timestamp: The time to match a frame to
    :param vid_timeseries: The time frames series to look at for a match
    :param report_acc: if set to 1, will report the accuracy of the match
    :return: index_of_lowest_diff , accuracy of match (if requested)
    """
    array = np.abs((vid_timeseries.to_numpy()) - timestamp)
    index_of_lowest_diff = np.argmin(array)
    if report_acc == 1:
        accuracy = abs(vid_timeseries[index_of_lowest_diff] - timestamp)
        return index_of_lowest_diff, accuracy
    else:
        return index_of_lowest_diff